In [9]:
!pip install pandas transformers accelerate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [1]:
from QA_model.qa import (
    load_csv_data_from_dirs,
    init_model,
    build_rag_data,
    rag_search,
    answer_question
)


/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
base_dir = "/data/jung/src/datas"  # CSV들이 들어있는 상위 폴더
df_all = load_csv_data_from_dirs(base_dir=base_dir)   # 모든 CSV 통합 로드
print(f"로딩된 CSV 행 개수: {len(df_all)}")


rag_data = build_rag_data(df_all) #아직 임베딩 벡터가 없으므로 임의로 모든 항목을 리스트에 저장해서 해당 리스트를 db 처럼 사용중
print(f"RAG 데이터 개수: {len(rag_data)}")

로딩된 CSV 행 개수: 1240
RAG 데이터 개수: 1240


In [4]:
# Qwen 사용시 질의 끝 혹은 코드 프롬프트에 한국어로 설명해달라는 프롬프트를 추가해야 함 아니면 중국어로만 나옴..
# model_handle = init_model("Qwen/Qwen2.5-7B-Instruct")
# 또는
model_handle = init_model("LGAI-EXAONE/EXAONE-3.5-7.8B-Instruct")
# 또는 (API 호출식)
# model_handle = init_model("GPT-4o-mini")   #GPT 이용 시에는 QA_model 폴더에 .env 파일 생성해야 함

Loading EXAONE model: LGAI-EXAONE/EXAONE-3.5-7.8B-Instruct


Loading checkpoint shards: 100%|██████████| 7/7 [00:08<00:00,  1.28s/it]


In [4]:
question_text = "카카오는 언제 비대면 주택담보대출을 출시했나요? 정답은 한국어로 대답해줘" 
answer = answer_question(question_text, rag_data, model_handle, top_k=3, use_bm25=False)
print(answer)


카카오뱅크는 2022년 2월에 비대면 주택담보대출 상품을 출시했습니다.


### validation 과 연결

#### csv 파일에 있는 쿼리에 대해 모두 응답하면서 점수 측정 후 저장하는 구조 

In [7]:
from evaluation.GPT_evaluation import evaluate
import pandas as pd
from QA_model.qa import bm25_rag_search


df = pd.read_csv("/data/jung/src/qa_validation_dataset_4_cleaning.csv")

query_list = df.apply(lambda row : [row["question"], row["answer"]], axis = 1).to_list()

relevant_items = rag_search(
        question=query_list[0][0],
        data_list=rag_data,
        top_k=3,
        use_bm25=False
        )

context_parts = []
for idx, item in enumerate(relevant_items, start=1):
    if item["summary"]:
        context_parts.append(f"{idx}. {item['summary']}")
    elif item["original_content"]:
        context_parts.append(f"{idx}. {item['original_content']}")
retrieved_contexts = "\n".join(context_parts)

print(retrieved_contexts)

generated_answer = answer_question(query_list[0][0], rag_data, model_handle, top_k=3, use_bm25=False)

eval_score = evaluate(query_list[0][0], retrieved_contexts, query_list[0][1], generated_answer)

print(eval_score)

1. SK하이닉스는 향후 3개년간 주주환원 정책을 공시하며, 연간 고정배당금을 1,500원으로 상향하고,  Free Cash Flow의 5% 재원을 재무 건전성 강화에 우선 활용한다고 밝혔다. 이는 메모리 업황 특성상 변동성이 큰 상황에서 내린 합리적인 결정이라고 판단된다.
2. SK하이닉스의 2025~2027년 주주환원 정책에 대한 기준과 변경 사항을 정리한 테이블로, 배당금 및 Free Cash Flow 활용 방안이 포함되어 있습니다.
3. SK하이닉스의 주요 재무 지표와 주식 관련 데이터를 나타내는 테이블로, KOSPI 지수, 시가총액, 외국인 지분율 등 다양한 정보를 포함하고 있습니다.
#### 1. Retrieval Evaluation (20 points):

1. **Do any of the retrieved contexts show strong similarity to the Ground Truth?** (5 points)  
   [Yes] - Justification: The first retrieved context closely mirrors the Ground Truth, mentioning the increase in fixed dividends to 1,500 won and the prioritization of using 5% of Free Cash Flow for financial stability.

2. **Do the retrieved contexts collectively capture essential information from the Ground Truth?** (5 points)  
   [Yes] - Justification: The first context captures the key changes in the shareholder return policy, while the second context provides a summary table that likely includes relevant detail

#### 추가 => 모델 클래스 분류 후 사용

In [1]:
from QA_model.qwen_model import QwenModel
from QA_model.exaone_model import ExaoneModel
from QA_model.gpt_model import GPTModel
from QA_model.utils import load_csv_data_from_dirs, build_rag_data

# 데이터 로드
base_dir = "/data/jung/src/datas"
df_all = load_csv_data_from_dirs(base_dir)
rag_data = build_rag_data(df_all)

# 모델 초기화 => 세개 중 선택
# model1 = QwenModel()
model2 = ExaoneModel()
# model3 = GPTModel(model_name="gpt-4o-mini")


/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 질문 및 답변
retrieval_results = "카카오뱅크는 2022년 2월에 비대면 주택담보대출 상품을 출시했습니다."
question = "카카오는 언제 비대면 주택담보대출을 출시했나요?"

prompt = f"주어진 문서를 반드시 참고하여 질문에 대한 답을 하세요.\n\n주어진 문서 : {retrieval_results}\n질문 : {question}"
# print(model1.answering(prompt))
# print(model2.answering(prompt))
print(model2.answering(prompt))

카카오는 2022년 2월에 비대면 주택담보대출 상품을 출시했습니다.


In [ ]:
import pandas as pd
from evaluation.GPT_evaluation import evaluate
from retrievals import bm25, dpr, ensemble
from QA_model.qwen_model import QwenModel
from QA_model.exaone_model import ExaoneModel
from QA_model.gpt_model import GPTModel
from DB.chromadb_storing import ChromaDB
from langchain.schema import Document
from utils import get_only_paragraphs, get_all_datas, create_documents

# Retriever 초기화 => DB 는 생성 안되어서 이건 되는 사람이 테스트좀..
topk = 3
DPRRetriever = dpr(db, topk=topk)
BM25Retriever = bm25(documents, topk=topk)
retrievals = [DPRRetriever, BM25Retriever]
weights = [0.5, 0.5]
search_type = 'mmr'
ensemble_retriever = ensemble(retrievals, topk=topk, weights=weights, search_type=search_type)

retrievers = {
    "BM25": BM25Retriever,
    "DPR": DPRRetriever,
    "Ensemble": ensemble_retriever
}

# 모델 초기화
models = {
    "QwenModel": QwenModel(),
    "ExaoneModel": ExaoneModel(),
    "GPTModel": GPTModel(model_name="gpt-4o-mini")
}

# 평가 데이터 로드
validation_df = pd.read_csv("/data/jung/src/qa_validation_dataset_4_cleaning.csv")
query_list = validation_df.apply(lambda row: [row["question"], row["answer"]], axis=1).to_list()

# 결과 저장을 위한 리스트
results = []

for model_name, model in models.items():
    for retriever_name, retriever in retrievers.items():
        for question, ground_truth in query_list:
            # Retrieval
            retrieved_docs = retriever.invoke(question)
            retrieved_texts = [doc.page_content for doc in retrieved_docs]  # Retrieved 텍스트만 사용
            retrieved_contexts = "\n".join(retrieved_texts)

            # 정답 생성
            prompt = f"주어진 문서를 반드시 참고하여 질문에 대한 답을 하세요.\n\n주어진 문서 : {retrieved_contexts}\n질문 : {question}"
            generated_answer = model.answering(prompt)

            # 평가
            eval_score = evaluate(question, retrieved_contexts, ground_truth, generated_answer)

            # 결과 저장
            results.append({
                "Model": model_name,
                "Retriever": retriever_name,
                "Question": question,
                "Ground Truth": ground_truth,
                "Generated Answer": generated_answer,
                "Retrieved Contexts": retrieved_contexts,
                "Total Retrieval Score": eval_score["retrieval_score"],
                "Total Score": eval_score["total_score"]
            })

# 결과를 DataFrame으로 변환
results_df = pd.DataFrame(results)

# 결과 CSV 파일 저장
results_df.to_csv("/data/jung/src/evaluation_results.csv", index=False, encoding="utf-8-sig")

print("Evaluation completed. Results saved to '/data/jung/src/evaluation_results.csv'.")
